In [31]:
%pip install -q transformers

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [2]:
# https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626

import numpy as np
import pandas as pd

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForQuestionAnswering

In [45]:
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa.head()

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


In [ ]:
#required columns in our dataframe
cols = ["text","question","answer"]
#list of lists to create our dataframe
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns=cols) 
#saving the dataframe to csv file for further loading
new_df.to_csv("CoQA_data.csv", index=False)

In [3]:
data = pd.read_csv("CoQA_data.csv")
data.head()

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project


In [4]:
print("Number of question and answers: ", len(data))

Number of question and answers:  108647


In [5]:
model_name = 'bert-large-cased-whole-word-masking-finetuned-squad'
# model_name = 'bert-base-uncased'

model = TFBertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = TFBertModel.from_pretrained(model_name)
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# encoded_input


All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-cased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [8]:
random_num = np.random.randint(0,len(data))

question = data["question"][0]
text = data["text"][0]

input_ids = tokenizer.encode(question, text)
tokens = tokenizer.convert_ids_to_tokens(input_ids)
# for token, id in zip(tokens, input_ids):
#     print('{:8}{:8,}'.format(token,id))
# tokenizer(question, text)

In [9]:
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)

num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)

num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)

SEP token index:  9
Number of tokens in segment A:  10
Number of tokens in segment B:  281


In [10]:
output = model(tokenizer(question, text, return_tensors='tf'))

In [11]:
answer_start = tf.math.argmax(output.start_logits[0])
answer_end = tf.math.argmax(output.end_logits[0])

if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    answer = "Answer not found"
    
print("Text:\n{}".format(text.capitalize()))
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Text:
The vatican apostolic library (), more commonly called the vatican library or simply the vat, is the library of the holy see, located in vatican city. formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. it has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. 

the vatican library is a research library for history, law, philosophy, science and theology. the vatican library is open to anyone who can document their qualifications and research needs. photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. 

in march 2014, the vatican library began an initial four-year project of digitising its collection of manuscripts, to be made available online. 

the vatican secret archives were separated from the library at the beginning of th